In [23]:
import intake
import json
import pystac
import s3fs
import helpers

In [45]:
timechunks = "06"
bucket = "nasa-eodc-scratch"
directory = f"NLDAS/netcdf/.timechunk{timechunks}"
filename = f"intake_{timechunks}.json"
catalog_url = f"s3://{bucket}/{directory}/{filename}"

In [46]:
catalog_url

's3://nasa-eodc-scratch/NLDAS/netcdf/.timechunk06/intake_06.json'

In [47]:
credentials = helpers.get_credentials()
s3fsfs = helpers.create_s3filesystem(credentials)

In [52]:
import xarray as xr
ds = xr.open_dataset(s3fsfs.open('s3://nasa-eodc-scratch/NLDAS/netcdf/.timechunk24/NLDAS_FOR0010_H.A20010101.030.beta.nc'), chunks={})
ds

<xarray.Dataset> Size: 58GB
Dimensions:  (lon: 11700, lat: 6500, time: 24)
Coordinates:
  * lon      (lon) float32 47kB -169.0 -169.0 -169.0 ... -52.03 -52.01 -52.0
  * lat      (lat) float32 26kB 7.005 7.015 7.025 7.035 ... 71.97 71.98 71.99
  * time     (time) datetime64[ns] 192B 2001-01-01 ... 2001-01-01T23:00:00
Data variables:
    Tair     (time, lat, lon) float32 7GB dask.array<chunksize=(24, 50, 90), meta=np.ndarray>
    Qair     (time, lat, lon) float32 7GB dask.array<chunksize=(24, 50, 90), meta=np.ndarray>
    PSurf    (time, lat, lon) float32 7GB dask.array<chunksize=(24, 50, 90), meta=np.ndarray>
    LWdown   (time, lat, lon) float32 7GB dask.array<chunksize=(24, 50, 90), meta=np.ndarray>
    SWdown   (time, lat, lon) float32 7GB dask.array<chunksize=(24, 50, 90), meta=np.ndarray>
    Wind_N   (time, lat, lon) float32 7GB dask.array<chunksize=(24, 50, 90), meta=np.ndarray>
    Wind_E   (time, lat, lon) float32 7GB dask.array<chunksize=(24, 50, 90), meta=np.ndarray>
    Rainf    (time, lat, lon) float32 7GB dask.array<chunksize=(24, 50, 90), meta=np.ndarray>
Attributes: (12/17)
    missing_value:          -9999.0
    time_definition:        hourly
    shortname:              NLDAS_FOR0010_H_3.0
    title:                  NLDAS Forcing Data L4 Hourly 0.01 x 0.01 degree V...
    version:                3.0 beta
    institution:            NASA GSFC
    ...                     ...
    websites:               https://ldas.gsfc.nasa.gov/nldas/v3/ ; https://li...
    MAP_PROJECTION:         EQUIDISTANT CYLINDRICAL
    SOUTH_WEST_CORNER_LAT:  7.005
    SOUTH_WEST_CORNER_LON:  -168.995
    DX:                     0.01
    DY:                     0.01

In [ ]:
with open('intake_06.json', 'r') as f:
    stac_dict = json.load(f)

# Parse as ItemCollection
item_collection = pystac.ItemCollection.from_dict(stac_dict)

In [82]:
item_collection

In [86]:
from shapely.geometry import Point, shape

def point_intersects_item_geometry(lon, lat, item: pystac.Item):
    """Check if a point intersects with a STAC item's geometry"""
    # Create a Point object
    point = Point(lon, lat)
    
    # Convert the GeoJSON geometry to a Shapely shape
    item_shape = shape(item.geometry)
    
    # Check if the point is within/intersects the shape
    return item_shape.contains(point)

In [101]:
def search_item_collection(collection: pystac.ItemCollection, latitude: float, longitude: float):
    filtered_items = [item for item in collection if point_intersects_item_geometry(longitude, latitude, item)]

    # Extract NetCDF assets
    netcdf_hrefs = []
    for item in filtered_items:
        for asset_key, asset in item.assets.items():
            if asset.media_type in ['application/netcdf', 'application/x-netcdf']:
                netcdf_hrefs.append(asset.href)
    
    # Load with xarray
    ds = xr.open_mfdataset([s3fsfs.open(href) for href in netcdf_hrefs], chunks={})
    
    return ds

In [ ]:
search_item_collection(item_collection, 60, -120)

# parquet explorations

In [67]:
#pip install stac-geoparquet

In [68]:
import stac_geoparquet

In [ ]:
record_batch_reader = stac_geoparquet.arrow.parse_stac_items_to_arrow(stac_dict['features'])
table = record_batch_reader.read_all()

In [79]:
stac_geoparquet.arrow.to_parquet(table, f'stac_{timechunks}.parquet')

In [80]:
import pyarrow.parquet as pq

In [81]:
pq.read_table('stac_06.parquet')

pyarrow.Table
assets: struct<data: struct<href: string, roles: list<element: string>, type: string>>
  child 0, data: struct<href: string, roles: list<element: string>, type: string>
      child 0, href: string
      child 1, roles: list<element: string>
          child 0, element: string
      child 2, type: string
bbox: struct<xmin: double, ymin: double, xmax: double, ymax: double>
  child 0, xmin: double
  child 1, ymin: double
  child 2, xmax: double
  child 3, ymax: double
geometry: binary
id: string
links: list<element: null>
  child 0, element: null
stac_extensions: list<element: null>
  child 0, element: null
stac_version: string
type: string
DX: double
DY: double
MAP_PROJECTION: string
SOUTH_WEST_CORNER_LAT: double
SOUTH_WEST_CORNER_LON: double
conventions: string
datetime: timestamp[us, tz=UTC]
doi: string
end_datetime: timestamp[us, tz=UTC]
history: string
institution: string
missing_value: double
reference: string
shortname: string
standard: string
start_datetime: timestamp